In [ ]:
import transformers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pandas as pd
from datasets import Dataset
import random
from PIL import Image, ImageDraw

In [ ]:
label_mapping = {
    0: 0,
    1: 1,
    2: 12,
    3: 23,
    4: 34,
    5: 38,
    6: 39,
    7: 40,
    8: 41,
    9: 42,
    10: 2,
    11: 3,
    12: 4,
    13: 5,
    14: 6,
    15: 7,
    16: 8,
    17: 9,
    18: 10,
    19: 11,
    20: 13,
    21: 14,
    22: 15,
    23: 16,
    24: 17,
    25: 18,
    26: 19,
    27: 20,
    28: 21,
    29: 22,
    30: 24,
    31: 25,
    32: 26,
    33: 27,
    34: 28,
    35: 29,
    36: 30,
    37: 31,
    38: 32,
    39: 33,
    40: 35,
    41: 36,
    42: 37
}

len(set(mappings.keys())), len(set(mappings.values()))

In [46]:
# Load the CSV file
test_df = pd.read_csv('./data/Test.csv')
test_df['labels'] = test_df['ClassId'].map(label_mapping)

# Create a Hugging Face Dataset from the DataFrame
test_dataset = Dataset.from_pandas(test_df)
from PIL import Image
from transformers import ViTFeatureExtractor

# Initialize the feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('cvetanovskaa/vit-base-patch16-224-in21k-gtsrb-tuned')

# Function to process the images
def transform(examples):
    # Adjust the path as necessary to locate the images
    images = [Image.open('./data/' + path).convert("RGB") for path in examples['Path']]
    # Apply feature extractor
    return feature_extractor(images, return_tensors="pt")

# Apply the transform function
test_dataset = test_dataset.map(transform, batched=True)

/Users/aleksandra.cvetanovska/.pyenv/versions/3.9.12/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Map:   0%|          | 0/12630 [00:00<?, ? examples/s]

In [47]:
test_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])

In [52]:
from transformers import ViTForImageClassification, ViTFeatureExtractor
from torch.utils.data import DataLoader
import torch

model = ViTForImageClassification.from_pretrained('cvetanovskaa/vit-base-patch16-224-in21k-gtsrb-tuned')

In [53]:
test_loader = DataLoader(test_dataset, batch_size=8)

# Move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Put the model in evaluation mode
model.eval()

# Store predictions and true labels
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        inputs = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [63]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9868566904196358


In [64]:
id2label = {
    0: 'Speed Limit 20Km/Hr',
    1: 'Speed Limit 30Km⁄Hr',
    2: 'No Passing Of  Vehicle Over 3.5Ton',
    3: 'Crossroads',
    4: 'Priority Road Sign',
    5: 'Yield',
    6: 'Stop',
    7: 'No Vehicles',
    8: 'Vehicle Over 3.5Ton Prohibited',
    9: 'No Entry',
    10: 'General Caution',
    11: 'Dangerous Curve To Left',
    12: 'Speed Limit 50Km⁄Hr',
    13: 'Dangerous Curve To Right',
    14: 'Double Curve',
    15: 'Bumpy Road',
    16: 'Slippery Road',
    17: 'Narrow Road On Right',
    18: 'Road Work',
    19: 'Traffic Sign',
    20: 'Pedestrian',
    21: 'Children Crossing',
    22: 'Bicycle Crossing',
    23: 'Speed Limit 60Km⁄Hr',
    24: 'Beware Of Ice/Snow',
    25: 'Wild Animal Crossing',
    26: 'End Of All Speed And Passing Limit',
    27: 'Compulsory Turn Right',
    28: 'Compulsory Turn Left',
    29: 'Straight Ahead',
    30: 'Go Straight Or Right',
    31: 'Go Straight Or Left',
    32: 'Avoid Obstacle On Right',
    33: 'Avoid Obstacle On Left',
    34: 'Speed Limit 70Km⁄Hr',
    35: 'Roundabout',
    36: 'End Of No Passing',
    37: 'End Of No Passing By Vehicle Over 3.5Ton',
    38: 'Speed Limit 80Km⁄Hr',
    39: 'End Of Speed Limit 80Km/Hr',
    40: 'Speed Limit 100Km⁄Hr',
    41: 'Speed Limit 120Km⁄Hr',
    42: 'No Passing'
}

print(classification_report(true_labels, predictions, target_names=id2label.values()))

                                          precision    recall  f1-score   support

                     Speed Limit 20Km/Hr       0.97      1.00      0.98        60
                     Speed Limit 30Km⁄Hr       0.99      1.00      1.00       720
      No Passing Of  Vehicle Over 3.5Ton       1.00      0.99      1.00       660
                              Crossroads       1.00      0.98      0.99       420
                      Priority Road Sign       1.00      0.99      1.00       690
                                   Yield       0.99      1.00      1.00       720
                                    Stop       1.00      1.00      1.00       270
                             No Vehicles       1.00      1.00      1.00       210
          Vehicle Over 3.5Ton Prohibited       1.00      1.00      1.00       150
                                No Entry       1.00      1.00      1.00       360
                         General Caution       1.00      0.96      0.98       390
               